In [1]:
import numpy as np
from numpy.fft import fft
import pandas as pd
import matplotlib.pyplot as plt
import zlib 
import scipy as sp

In [2]:
import pandas as pd

df = pd.read_csv('csv/Turntable1.csv')

In [3]:
t = df[['Time']].to_string()
accX = df[['Accel X (g)']].to_string()
temp = df[['Temp']].to_numpy()
gyroZ = df[['Gyro Z (dps)']].to_numpy()
gravY = df[['Grav Y']].to_numpy()

In [4]:
with open('accX.txt', 'w') as f:
    f.writelines(accX)

In [5]:
import zlib
import time 

def compress(plaintext, level):
    comp = zlib.compress(plaintext.encode('utf-8'), level)
    return comp

def decompress(comp):
    decomp = zlib.decompress(comp)
    return decomp

' '

## Compression

In [6]:
def compress(plaintext, level):
    comp = zlib.compress(plaintext.encode('utf-8'), level)
    return comp

In [7]:
with open('csv_as_text/WalkingAround.txt') as f:
    original = f.readlines()


In [23]:
data = "hi"

In [24]:
comp = compress(data, 8)

with open('SerialData/comp_sample.txt', 'w') as f:
    f.write(str(comp))

In [5]:
compressed = []
for line in original:
    comp = compress(line, 8)
    compressed.append(comp)

In [6]:
with open('compressed/WalkingAround_compressed.txt', 'wb') as f:
    f.writelines(compressed)

In [7]:
decompressed = []
for line in compressed:
    decomp = zlib.decompress(line)
    decompressed.append(decomp)

In [8]:
with open('decompressed/WalkingAround_decompressed.txt', 'wb') as f:
    f.writelines(decompressed)

In [9]:
with open('csv_as_text/WalkingAround.txt') as f:
    original_file = f.read()

with open('decompressed/WalkingAround_decompressed.txt') as f:
    decompressed_file = f.read()

In [10]:
if original_file==decompressed_file:
    print('All data retained after compression and decompression')
else:
    print('Insufficient data retained after compression and decompression')

All data retained after compression and decompression


## Encryption

In [207]:
from Crypto.Cipher import AES
from secrets import token_bytes
import time

#generate key
key = token_bytes(16)

#encrypt function
def encrypt(msg):
    cipher = AES.new(key, AES.MODE_EAX)
    nonce = cipher.nonce
    ciphertext, tag = cipher.encrypt_and_digest(msg.encode('ascii'))
    return nonce, ciphertext, tag

#decrypt function
def decrypt(nonce, ciphertext, tag):
    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
    plaintext = cipher.decrypt(ciphertext)
    try:
        cipher.verify(tag)
        return plaintext.decode('ascii')
    except:
        return False

#function to check if decrypted data is the same as the original data
def check_decryption(plaintext, original):
    if plaintext == original:
        print('Decryption Successful')
    else:
        print('Decryption Failed')

#timing function
def get_time():
    t = time.perf_counter()
    return t

def do_encryption(filenames, timing=False, generate_files=False):
    for file in filenames:
        #read text file as a string
        with open('csv_as_text/'+file) as f:
            original = f.read()
    
        start = get_time()
        #encrypt data
        nonce, ciphertext, tag = encrypt(original)
        stop = get_time()
        encrypt_time = round(stop-start, 4) 
        
        start = get_time()
        #decrypt data
        plaintext = decrypt(nonce, ciphertext, tag)
        stop = get_time()
        decrypt_time = round(stop-start, 4)

        total_time = encrypt_time+decrypt_time

        #generate textfiles
        if generate_files:
            with open('encrypted/encrypted_'+file+'.txt', 'w') as f:
                f.writelines(str(ciphertext))

            start = get_time()
            with open('decrypted/decrypted_'+file+'.txt', 'w') as f:
                f.writelines(str(plaintext))
            stop = get_time()
            write_time = round(stop-start, 4)
            total_time += write_time

        print(f'Checking Decryption for {file}')
        check_decryption(plaintext, original)

        if timing:
            print(f'Encrption Execution Time {encrypt_time} seconds')
            print(f'Decryption Execution Time {decrypt_time} seconds')
            if generate_files:
                print(f'Time to write to text file {write_time} seconds')
            print(f'Total Execution Time {total_time} seconds')

        print('-------------------------------------')